# Imports

In [ ]:
import numpy as np
import pandas as pd

from lightgbm import LGBMClassifier, early_stopping, log_evaluation

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder

import gc

In [ ]:
class CONFIG:
    random_state = 69420
    kaggle = True
    kaggle_path = '../input/amexfeather/'
    local_path = ''

# Data preprocessing

In [ ]:
%%time
train = pd.read_feather(CONFIG.kaggle_path+'train_data.ftr')
train.head()

In [ ]:
%%time
# Keep the last statement month per customer
# https://www.kaggle.com/competitions/amex-default-prediction/discussion/327094
train =  (train
            .groupby('customer_ID')
            .tail(1)
            .set_index('customer_ID', drop=True)
            .sort_index()
            .drop(['S_2'], axis='columns'))

In [ ]:
_ = gc.collect()

In [ ]:
train.shape

In [ ]:
all_cols = train.columns.to_list()

cat_cols = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']

num_cols = [col for col in all_cols if col not in cat_cols + ["target"]]

In [ ]:
X = train[cat_cols + num_cols]
y = train['target']

X.shape, y.shape

In [ ]:
%%time
enc = OrdinalEncoder()
X[cat_cols] = enc.fit_transform(X[cat_cols])
_ = gc.collect()

## Competition metric

In [ ]:
def amex_metric(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:

    def top_four_percent_captured(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        four_pct_cutoff = int(0.04 * df['weight'].sum())
        df['weight_cumsum'] = df['weight'].cumsum()
        df_cutoff = df.loc[df['weight_cumsum'] <= four_pct_cutoff]
        return (df_cutoff['target'] == 1).sum() / (df['target'] == 1).sum()
        
    def weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        df['random'] = (df['weight'] / df['weight'].sum()).cumsum()
        total_pos = (df['target'] * df['weight']).sum()
        df['cum_pos_found'] = (df['target'] * df['weight']).cumsum()
        df['lorentz'] = df['cum_pos_found'] / total_pos
        df['gini'] = (df['lorentz'] - df['random']) * df['weight']
        return df['gini'].sum()

    def normalized_weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        y_true_pred = y_true.rename(columns={'target': 'prediction'})
        return weighted_gini(y_true, y_pred) / weighted_gini(y_true, y_true_pred)

    g = normalized_weighted_gini(y_true, y_pred)
    d = top_four_percent_captured(y_true, y_pred)

    return 0.5 * (g + d)

# Model Training

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
clf = LGBMClassifier(
    n_estimators=50000,
    device='gpu',
    random_state=CONFIG.random_state,
    extra_trees=True
)

In [ ]:
%%time
clf.fit(
    X_train, y_train, 
    eval_set=[(X_test,y_test)],
    callbacks=[early_stopping(50), log_evaluation(0)]
)

In [ ]:
y_pred = pd.DataFrame(y_test.copy(deep=True))
y_pred = y_pred.rename(columns={'target':'prediction'})
y_pred

In [ ]:
%%time
y_pred["prediction"] = clf.predict_proba(X_test)[:,1]

In [ ]:
y_pred

In [ ]:
y_test = pd.DataFrame(y_test)

In [ ]:
%%time
amex_metric(y_test, y_pred)

# Submission

In [ ]:
del train, X, y, X_test, X_train, y_train, y_test, y_pred
_ = gc.collect()

In [ ]:
%%time
test = pd.read_feather(CONFIG.kaggle_path+'test_data.ftr')
test.head()

In [ ]:
test =  (
    test
    .groupby('customer_ID')
    .tail(1)
    .set_index('customer_ID', drop=True)
    .sort_index()
    .drop(['S_2'], axis='columns')
)

In [ ]:
_ = gc.collect()

In [ ]:
%%time
test[cat_cols] = enc.transform(test[cat_cols])
_ = gc.collect()

In [ ]:
test["prediction"] = clf.predict_proba(test[cat_cols + num_cols])[:,1]
test.head()

In [ ]:
test["prediction"].to_csv("submission.csv", index=True)